# 要約 
このJupyter Notebookでは、「LMSYS - Chatbot Arena」コンペティションに参加するためのモデルを構築しています。具体的には、大規模言語モデル（LLM）を用いて、異なるチャットボットの応答を比較し、どちらの応答が好まれるかを予測するタスクに取り組んでいます。

### 問題に取り組んでいる内容
Notebookは、人間によるチャットボットの応答の選択に基づいて、どのモデルがより好ましいかを予測するために設計されています。トレーニングデータには、ユーザーが選んだ応答が含まれています。そのため、各応答が勝つ確率を計算し、それに基づいて予測を行います。

### 使用している手法やライブラリ
1. **ライブラリ**
   - `transformers`: Hugging Faceのライブラリを使用し、DeBERTa V3モデルを含む事前学習済みモデルを利用してシーケンス分類タスクを処理。
   - `datasets`: データを簡単に処理するためのライブラリ。
   - `sklearn`: モデル評価とデータ処理に利用。
   - `torch`: PyTorchを使用してニューラルネットワークを構築し、訓練します。

2. **主要な手法**
   - **データ前処理**: トークナイゼーションを行い、モデルが理解できる形式にデータを変換します。
   - **モデル構築**: DeBERTa V3モデルを使用して、分類タスクとして設定。
   - **トレーニング**: Stratified K-Fold交差検証を用いて、モデルの訓練と検証を行い、最良モデルを選択します。
   - **評価指標**: ログ損失(log loss)と精度(accuracy)を用いてモデルのパフォーマンスを評価します。

3. **訓練および推論設定**
   - 訓練と評価のためのハイパーパラメータ（学習率やバッチサイズなど）を設定し、混合精度訓練を行います。
   - 実際のデータでトレーニングを行った後、テストデータを使ってモデルが生成した予測を取得し、結果をCSVファイルとして保存します。

このNotebookは、データの読み込みからモデルの訓練、推論、結果の保存までの全過程をカバーしており、コンペティション参加者が迅速にモデルを構築し、提出できるようになっています。

---


# 用語概説 
以下は、Jupyter Notebook内のコードやドキュメントに出てくる専門用語の解説です。特に初心者がつまずきそうな部分や、このノートブック特有の用語に焦点を当てています。

### 専門用語の解説

1. **StratifiedKFold**:
   - クロスバリデーションの一手法で、データセットのラベル分布がそれぞれのフォールドに均等に分配されるように訓練データを分ける方法。クラスの不均衡問題を軽減するのに役立つ。

2. **AutoTokenizer**:
   - Hugging FaceのTransformersライブラリにおけるクラスで、事前学習済みのトークナイザーを自動で読み込むためのもの。テキストをトークン（数値の列）に変換するために使われる。

3. **attention_mask**:
   - トークン化された入力データに含まれる、どのトークンが意味を持ち、どれがパディングであるかを示すバイナリマスク。1は有効なトークン、0はパディングトークンを示す。

4. **DataCollatorWithPadding**:
   - データローダの一部で、バッチ内のデータの長さを揃えるためにパディングを適用する役割を持つコンポーネント。これにより、ミニバッチが同じ形状を持つようになる。

5. **log_loss**:
   - モデルの出力確率と実際のラベルとの間の誤差を測る指標。特にクラス確率のモデル評価に使われる。値が小さいほどモデルの予測が良いことを示す。

6. **fp16 (半精度浮動小数点)**:
   - モデルの訓練において使用されるデータ型の一つで、通常の32ビット浮動小数点数（fp32）よりもメモリを少なくすることができ、計算速度も向上する。一部のハードウェアでは特に効率的。

7. **EarlyStoppingCallback**:
   - モデルの訓練中に、指定された回数（patience）で評価メトリックが改善しない場合に訓練を停止するための仕組み。オーバーフィッティングを防ぐのに役立つ。

8. **num_labels**:
   - モデルのアウトプット層において予測可能なクラスの数を指定するためのパラメータ。バイナリ分類や多クラス分類の場合に、それぞれのクラス数に設定される。

9. **weight_decay**:
   - モデルを訓練する際に、重みの大きさを抑え、過学習を防ぐためのペナルティの設定。L2正則化とも呼ばれ、設定された値に比例して重みを減衰させる。

10. **train_fraction**:
    - データセット全体の中で、トレーニングに使用するデータの割合を指定するパラメータ。バリデーションやテストデータとの適切な分割を行うために使われる。

これらの用語が理解できるようになると、Notebookの内容をより良く把握し、実践的な機械学習モデルの開発に役立つでしょう。

---


In [ ]:
import os
import logging
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import Dataset
from sklearn.metrics import log_loss
import torch
from functools import partial
import warnings
from transformers import logging as transformers_logging
from transformers import EarlyStoppingCallback
import json
from pprint import pformat
from tqdm import trange
warnings.simplefilter('ignore')

TYPE = "large"
VER= 14
DATE = "0717"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

# ロギングの設定
transformers_logging.set_verbosity_error()
logging.basicConfig(level=logging.INFO, filename=f'logs_v{VER}.log', filemode='a',
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


class PATHS:
    train_path = '/kaggle/input/lmsys-chatbot-arena/train.csv'  # トレーニングデータのパス
    test_path = '/kaggle/input/lmsys-chatbot-arena/test.csv'    # テストデータのパス
    sub_path = '/kaggle/input/lmsys-chatbot-arena/sample_submission.csv'  # 提出サンプルのパス
    model_name = f"deberta-v3-{TYPE}"  # モデル名
    model_path = f"/root/autodl-tmp/ase2/huggingfacedebertav3variants/{model_name}"  # モデルの保存パス
    tokenizer_path = f"/kaggle/input/lmsys-{TYPE}{VER}-{DATE}/fold_0/tokenizer"  # トークナイザーのパス
    general_tokenizer = "/kaggle/input/lmsys-base4-0704/fold_0/tokenizer"  # 一般的なトークナイザーのパス

class CFG:
    seed = 42  # 乱数シード
    max_length = 512  # 最大入力長
    lr = 5e-5  # 学習率
    weight_decay = 0.01  # 重み減衰
    warmup_ratio = 0 # 学習率ウォームアップ比率
    max_grad_norm = 1000  # 最大勾配ノルム
    lr_scheduler_type = 'linear'  # 学習率スケジュールタイプ
    frozen_embedding = False # 埋め込み層を凍結するかどうか
    frozen_num = 6  # 凍結する層の数
    train_batch_size = 32  # 訓練バッチサイズ
    eval_batch_size = 64  # 評価バッチサイズ
    evaluation_strategy = 'steps'  # 評価戦略をステップに設定
    metric_for_best_model = "eval_log_loss"  # 最良モデル選択のためのメトリック
    save_strategy = 'steps'  # ステップごとに保存
    save_steps = 200  # 200ステップごとにモデルを保存
    save_total_limit = 1  # 保存されるチェックポイントの合計数制限
    train_epochs = 5  # 訓練エポック数
    num_labels = 6  # ラベルの数
    output_dir = f'/kaggle/input/lmsys-{TYPE}{VER}-{DATE}'  # 出力ディレクトリ
    fp16 = True  # 混合精度訓練を使用
    load_best_model_at_end = True  # 訓練終了時に最良モデルをロード
    report_to = 'none'  # 外部ツールに訓練ログを報告しない
    optim = 'adamw_torch'  # オプティマイザタイプ
    logging_first_step = True  # 最初のステップのログを記録する
    logging_steps = 200  # 200ステップごとにログを記録
    logging_dir =f'logs_v{VER}'  # ログ保存ディレクトリ
    n_splits = 5  # クロスバリデーションの分割数
    model_name = PATHS.model_name  # モデル名
    greater_is_better = False  # メトリックが大きいほど良いかどうか
    early_stop = False  # 早期停止を使用するか
    early_stopping_patience = 3  # 改善なしでモデルの訓練を停止する評価の呼び出し回数
    early_stopping_threshold = 0.001  # 改善と見なすための最小変化量

def seed_everything(seed):
    import random
    import os
    import numpy as np
    import torch
    
    random.seed(seed)  # Pythonの乱数シードを設定
    os.environ['PYTHONHASHSEED'] = str(seed)  # ハッシュシードを設定
    np.random.seed(seed)  # NumPyの乱数シードを設定
    torch.manual_seed(seed)  # PyTorchの乱数シードを設定
    torch.cuda.manual_seed(seed)  # CUDAの乱数シードを設定
    torch.backends.cudnn.deterministic = True  # 再現性を持たせる

seed_everything(seed=CFG.seed)  # 乱数シードの設定

# tokenizer = AutoTokenizer.from_pretrained(PATHS.tokenizer_path)
tokenizer = AutoTokenizer.from_pretrained(PATHS.general_tokenizer)  # トークナイザーの初期化
sep_token = tokenizer.sep_token_id  # セパレータトークンのIDを取得

def log_parameters(logger):
    """PATHSとCFGクラスからすべてのパラメータをログに記録します。"""
    logger.info("=== パラメータ設定 ===")
    
    logger.info("PATHS:")
    for key, value in PATHS.__dict__.items():
        if not key.startswith('__'):
            logger.info(f"  {key}: {value}")  # 各パラメータをログに記録
    
    logger.info("CFG:")
    for key, value in CFG.__dict__.items():
        if not key.startswith('__'):
            logger.info(f"  {key}: {value}")  # 各パラメータをログに記録
    
    logger.info("=*100")

def tokenize_function(row, tokenizer):
    max_len = CFG.max_length - 2  # セパレータトークンのために2を引く
    tokens_prompt = tokenizer(row['prompt'], truncation=True, max_length=max_len//4, add_special_tokens=False)['input_ids']  # プロンプトをトークン化
    remaining_length = max_len - len(tokens_prompt)  # 残りの長さを計算
    
    tokens_response_a = tokenizer(row['response_a'], truncation=True, max_length=remaining_length//2, add_special_tokens=False)['input_ids']  # 応答Aをトークン化
    remaining_length -= len(tokens_response_a)  # 残りの長さを更新
    tokens_response_b = tokenizer(row['response_b'], truncation=True, max_length=remaining_length, add_special_tokens=False)['input_ids']  # 応答Bをトークン化
    
    input_ids = [tokenizer.cls_token_id] + tokens_prompt + [sep_token] + tokens_response_a + [sep_token] + tokens_response_b  # 入力IDを作成
    token_type_ids = [0] * (len(tokens_prompt) + 2) + [1] * (len(tokens_response_a) + 1) + [2] * len(tokens_response_b)  # トークンタイプIDsを作成
    attention_mask = [1] * len(input_ids)  # アテンションマスクを作成
    
    padding_length = CFG.max_length - len(input_ids)  # パディングの長さを計算
    if padding_length > 0:
        input_ids = input_ids + [tokenizer.pad_token_id] * padding_length  # パディングを追加
        token_type_ids = token_type_ids + [0] * padding_length  # パディングを追加
        attention_mask = attention_mask + [0] * padding_length  # パディングを追加
    
    return {
        'input_ids': input_ids[:CFG.max_length],  # 最大長に制限する
        'token_type_ids': token_type_ids[:CFG.max_length],  # 最大長に制限する
        'attention_mask': attention_mask[:CFG.max_length],  # 最大長に制限する
    }

def add_label(df):
    labels = np.zeros(len(df), dtype=np.int32)  # ラベルの配列を作成
    labels[df['winner_model_a'] == 1] = 0  # モデルAが勝った場合のラベル
    labels[df['winner_model_b'] == 1] = 1  # モデルBが勝った場合のラベル
    labels[df['winner_tie'] == 1] = 2  # 引き分けの場合のラベル
    df['labels'] = labels  # データフレームにラベルを追加
    return df

def process_data(df, mode='train'):
    dataset = Dataset.from_pandas(df)  # データフレームからデータセットを作成
    tokenized_dataset = dataset.map(partial(tokenize_function, tokenizer=tokenizer), batched=False)  # データセットをトークン化
    remove_cols = ['id', 'prompt', 'response_a', 'response_b']  # 削除する列を指定
    if mode == 'train':
        remove_cols += ['model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie']  # トレーニングモードの場合、さらに列を削除
    tokenized_dataset = tokenized_dataset.remove_columns(remove_cols)  # 指定した列を削除
    return tokenized_dataset

def split_train_val(dataset, train_fraction):
    np.random.seed(0)  # 乱数シードを設定
    ixs = np.arange(len(dataset))  # データセットのインデックスを生成
    cutoff = int(len(ixs) * train_fraction)  # 訓練データのカットオフポイント
    np.random.shuffle(ixs)  # インデックスをシャッフル
    ixs_train = ixs[:cutoff]  # 訓練用インデックス
    ixs_val = ixs[cutoff:]  # 検証用インデックス
    fit_train = dataset.select(ixs_train)  # 訓練データを選択
    fit_val = dataset.select(ixs_val)  # 検証データを選択
    return fit_train, fit_val


def compute_metrics(eval_pred):
    logits, labels = eval_pred  # 評価予測からロジットとラベルを取得
    probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)  # 確率を計算
    return {
        'eval_log_loss': log_loss(labels, probabilities),  # ログ損失を計算
        'eval_accuracy': (np.argmax(logits, axis=1) == labels).mean()  # 精度を計算
    }
    
def train_model():
    log_parameters(logger)  # パラメータをログに記録
    train_df = pd.read_csv(PATHS.train_path)  # トレーニングデータを読み込み
    train_df = add_label(train_df)  # ラベルを追加
    train_tokenized = process_data(train_df, mode='train')  # トークン化されたデータを取得
    
    skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)  # ストラティファイドKフォールドを作成
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(train_tokenized, train_tokenized['labels'])):  # 各フォールドでトレーニングと検証インデックスを分割
        print(f"フォールド {fold + 1} のトレーニング")  # フォールド番号を出力
        logger.info(f"フォールド {fold + 1} のトレーニング")  # フォールド番号をログに記録
        
        fit_train = train_tokenized.select(train_idx)  # 訓練データを選択
        fit_val = train_tokenized.select(val_idx)  # 検証データを選択
        
        model = AutoModelForSequenceClassification.from_pretrained(  # 事前学習済みモデルを読み込み
            PATHS.model_path,
            num_labels=3,  # ラベル数を指定
            problem_type="single_label_classification"  # 問題タイプを指定
        )
        
        training_args = TrainingArguments(  # 訓練引数を設定
            output_dir=f"{CFG.output_dir}/fold_{fold}",  # モデルとチェックポイントの出力ディレクトリ
            fp16=CFG.fp16,  # 混合精度訓練を使用
            learning_rate=CFG.lr,  # 学習率
            per_device_train_batch_size=CFG.train_batch_size,  # 各デバイスでの訓練バッチサイズ
            per_device_eval_batch_size=CFG.eval_batch_size,  # 各デバイスでの評価バッチサイズ
            num_train_epochs=CFG.train_epochs,  # トレーニングのエポック数
            weight_decay=CFG.weight_decay,  # 重み減衰
            evaluation_strategy=CFG.evaluation_strategy,  # 評価戦略
            metric_for_best_model=CFG.metric_for_best_model,  # 最良モデルを選択するためのメトリック
            save_strategy=CFG.save_strategy,  # 保存戦略
            save_total_limit=CFG.save_total_limit,  # チェックポイントの総保存数制限
            load_best_model_at_end=CFG.load_best_model_at_end,  # 最良モデルを訓練終了時にロード
            report_to=CFG.report_to,  # ログを外部ツールに報告しない
            warmup_ratio=CFG.warmup_ratio,  # 学習率のウォームアップ比率
            lr_scheduler_type=CFG.lr_scheduler_type,  # 学習率スケジューラーのタイプ
            optim=CFG.optim,  # 使用するオプティマイザのタイプ
            logging_first_step=CFG.logging_first_step,  # 最初のステップのログを記録
            greater_is_better=CFG.greater_is_better,  # メトリックが大きいほど良いかどうか
            
            # max_grad_norm=CFG.max_grad_norm,  # 勾配クリッピングの設定
            
            logging_steps=CFG.logging_steps,  # 200ステップごとにログを記録
            logging_dir =f'logs_v{VER}',  # ログ保存ディレクトリ
        
            save_steps=CFG.save_steps,  # 200ステップごとにモデルを保存
            eval_steps=CFG.save_steps,  # eval_stepsパラメータを追加、save_stepsと一致させる
        )

         # 訓練引数をログに記録
        logger.info("訓練引数:")
        logger.info(pformat(training_args.to_dict()))

        if CFG.frozen_embedding:  # 埋め込み層を凍結する設定の場合
            n = CFG.frozen_num
            # 埋め込み層を凍結
            for i, layer in enumerate(model.deberta.encoder.layer[:n]):
                for param in layer.parameters():
                    param.requires_grad = False  # 訓練から除外
            for param in model.deberta.embeddings.parameters():
                param.requires_grad = False  # 訓練から除外

        # トークナイザーの初期化
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # データコラトレーターの初期化

        # EarlyStoppingCallbackの作成
        if CFG.early_stop:  # 早期停止を使用する設定の場合
            early_stopping_callback = EarlyStoppingCallback(
                early_stopping_patience=CFG.early_stopping_patience,  # 訓練停止の耐久期間
                early_stopping_threshold=CFG.early_stopping_threshold,  # 改善と見なす最小変化
            )
        
            trainer = Trainer(  # トレーナーの初期化
                model=model,
                args=training_args,
                train_dataset=fit_train,  # 訓練データセット
                data_collator=data_collator,  # データコラトレーター
                eval_dataset=fit_val,  # 検証データセット
                compute_metrics=compute_metrics,  # メトリックの計算関数
                callbacks=[early_stopping_callback],  # 早期停止コールバックを追加
            )
        else:
            trainer = Trainer(  # トレーナーの初期化
                model=model,
                args=training_args,
                train_dataset=fit_train,  # 訓練データセット
                data_collator=data_collator,  # データコラトレーター
                eval_dataset=fit_val,  # 検証データセット
                compute_metrics=compute_metrics,  # メトリックの計算関数
            )
        
        trainer.train()  # モデルの訓練を開始
        
        # モデルを保存
        trainer.save_model(f"{CFG.output_dir}/fold_{fold}/best_model")  # 最良モデルを保存
        tokenizer.save_pretrained(f"{CFG.output_dir}/fold_{fold}/tokenizer")  # トークナイザーを保存
        
        # 結果をログに記録
        eval_result = trainer.evaluate()  # 評価を実施
        logger.info(f"フォールド {fold + 1} - 評価結果: {eval_result}")  # 評価結果をログに記録
        logger.info("=*100")

def predict_test():
    test_df = pd.read_csv(PATHS.test_path)  # テストデータを読み込み
    test_tokenized = process_data(test_df, mode='test')  # トークン化されたテストデータを取得
    
    predictions = []  # 予測結果を格納するリスト
    
    for fold in trange(CFG.n_splits):  # 各フォールドで予測を行う
        model = AutoModelForSequenceClassification.from_pretrained(f"{CFG.output_dir}/fold_{fold}/best_model")  # 最良モデルを読み込み
        model.eval()  # 評価モードに設定
        
        trainer = Trainer(model=model)  # トレーナーの初期化
        fold_preds = trainer.predict(test_tokenized).predictions  # テストデータに対する予測を取得
        fold_preds = np.exp(fold_preds) / np.sum(np.exp(fold_preds), axis=1, keepdims=True)  # 確率に変換
        predictions.append(fold_preds)  # フォールドの予測結果を追加
    
    # フォールド間の予測を平均化
    final_preds = np.mean(predictions, axis=0)  # 最終予測を計算
    display(predictions)  # 予測結果を表示
    logger.info(f"最終予測: {final_preds}")  # 最終予測をログに記録
    
    # 提出ファイルの作成
    submission = pd.DataFrame({
        'id': test_df['id'],  # テストデータのID
        'winner_model_a': final_preds[:, 0],  # モデルAの勝者確率
        'winner_model_b': final_preds[:, 1],  # モデルBの勝者確率
        'winner_tie': final_preds[:, 2]  # 引き分け確率
    })
    
    submission.to_csv('submission.csv', index=False)  # 提出ファイルをCSV形式で保存
    display(submission)  # 提出結果を表示

In [ ]:
%time  # コードの実行時間を計測します
if __name__ == "__main__":  # このスクリプトがメインプログラムとして実行されている場合
#     train_model()  # モデルを訓練する関数を呼び出す（コメントアウト中）
    predict_test()  # テストデータに対する予測を行う関数を呼び出す

In [ ]:
# トークナイザーライブラリをインポートします
# print(tokenizers.__version__)  # トークナイザーライブラリのバージョンを出力します

---

# コメント

> ## Rise_Hand
> 
> いつも素晴らしい仕事だね、兄弟。
> 
> 

---

> ## Yuxi Xue
> 
> 素晴らしい仕事！！！！！
> 
> 

---

> ## Korey Ma
> 
> いい仕事だね！！！🥳
> 
> 

---

> ## YEI0907
> 
> 兄弟、推論にどのくらい時間がかかりましたか？
> 
> 
> > ## Roschild.Rui（トピック作成者）
> > 
> > だいたい2〜3時間くらいだと思います。
> > 
> > 
> 

---